# Image stack packager

This tool packages data. The data a stack of images off of a brightfield microscope which reside in an Allen Institute repository, the Cell Types Database. This tool downloads a stack of images and generates a JSON manifest of the files. All output artifacts are files in a single directory.

This notebook has been tested to work on Google's Colab and vanilla JupyterLab.


## Setup

The Allen Institute maintains the Allen SDK, `allensdk`, for accessing their data products. Colab has tons of Python packages pre-installed but `allensdk` is not one of them. For more details about working with `allensdk` on Colab, see Reconstrue's [AllenSDK on Colab](http://reconstrue.com/data_sources/allen_institute/allensdk_on_colab.html). For now, simply install the SDK.

In [0]:
!pip3 --quiet install allensdk 

In [0]:
import json

## Methods of access

The images can be accessed through a web UI and/or programatically in Python.

### brain-map.org web UI

[brain-map.org](http://brain-map.org) is where the target images reside. The repository has a web UI, wherein the image stack can be viewed. Here is an example from their documentation [[*](http://help.brain-map.org/display/celltypes/Physiology+and+Morphology)]:

>displays two orthogonal projections of the biocytin filled neuron and the neuron's 3D morphology reconstruction. From this page, you can also view the stack of high resolution images used for the reconstruction.

![](http://help.brain-map.org/download/attachments/8323624/MorphBrowse.PNG?version=1&modificationDate=1476664307214&api=v2)

So, we can explore the web UI to preview what the images look like but we want to download them via Python code:

> You can also access the data programatically and obtain sample code to run your own model simulations. For more details go to the Download page. 



### RESTful RMA

The Allen Institute first came up with a RESTful interface to their resources, called [RMA](http://help.brain-map.org/pages/viewpage.action?pageId=5308449). RMA is a [HATEOAS](https://restfulapi.net/hateoas/) style RESTful API.


The `allensdk` is Python code which provides a programmatic interface to the info available via RMI. It also maintains a cache of files for performance purposes (`allensdk.core.cell_types_cache.CellTypesCache`).

Although `allensdk` can provide metadata about cells in the repository, it does not have methods to acquire the raw image stack. To get the raw images, RMI is the only method. So, `allensdk` can provide IDs of available cells, further work is required to then iterate through the stack and grab each file.



#### Probing RMA

Their documentation includes [example URLs for fetching data](http://help.brain-map.org/display/celltypes/API#API-morphology_image_download). Here are some of those exercised in a Jupyter context.

Seemingly data can be requested in multiple formats: XML, JSON, and CSV.


##### As XML

In [0]:
xml_request_url = "http://api.brain-map.org/api/v2/data/query.xml?criteria=model::ProjectionImage,rma::criteria,[specimen_id$eq313862022]"
xml_file_name = "response.xml"
!wget -O {xml_file_name} {xml_request_url}

--2020-03-26 20:05:48--  http://api.brain-map.org/api/v2/data/query.xml?criteria=model::ProjectionImage,rma::criteria,[specimen_id]
Resolving api.brain-map.org (api.brain-map.org)... 63.237.233.29
Connecting to api.brain-map.org (api.brain-map.org)|63.237.233.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: ‘response.xml’

response.xml            [ <=>                ]  47.38K   278KB/s    in 0.2s    

2020-03-26 20:05:48 (278 KB/s) - ‘response.xml’ saved [48518]



In [0]:
!cat {xml_file_name}

That's nice: for each image stack, they provide both MaximumIntensityProjection and MinimumIntensityProjection from both the frontal view plane (xy) and one of the two side views (yz plane).

##### As JSON

This is exactly the same as the above XML response, except in the requested URL `/query.xml?` is changed to `/query.json?` 

In [0]:
json_quest_url = "http://api.brain-map.org/api/v2/data/query.json?criteria=model::ProjectionImage,rma::criteria,[specimen_id$eq313862022]"
json_file_name = "/content/response.json"

!wget -O {json_file_name}

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [0]:
with open(json_file_name) as f:
  eg_data = json.load(f)

print(json.dumps(eg_data, indent=2))

{
  "success": true,
  "id": 0,
  "start_row": 0,
  "num_rows": 50,
  "total_rows": 2686,
  "msg": [
    {
      "annotated": false,
      "axes": "xy",
      "bits_per_component": 8,
      "data_set_id": null,
      "expression": null,
      "expression_path": null,
      "failed": false,
      "height": 7592,
      "id": 520209042,
      "image_height": 7592,
      "image_type": "MinimumIntensityProjection - xy",
      "image_width": 7596,
      "isi_experiment_id": null,
      "lims1_id": null,
      "number_of_components": 1,
      "ophys_experiment_id": null,
      "path": "/external/mousecelltypes/prod765/specimen_517330781/min_xy_517330781.aff",
      "projection_function": "min",
      "resolution": 0.1144,
      "section_number": 0,
      "specimen_id": 517330781,
      "structure_id": null,
      "tier_count": 6,
      "width": 7596,
      "x": 0,
      "y": 0
    },
    {
      "annotated": false,
      "axes": "yz",
      "bits_per_component": 8,
      "data_set_id": null,
